<h1><center> Protein classification using Random Forest </center></h1>

Using the new dataset with environment features

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import pbs
import sys
import math
import time
import random

from tqdm import tqdm
from pprint import pprint
from pathlib import Path

from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, auc
from sklearn.metrics import f1_score, classification_report


import seaborn as sns; sns.set(style='ticks', font='serif')

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

C:\Users\Acer\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
data_dir = "C:/Users/Acer/Documents/kriti/random_forests/training_k3/"

In [3]:
df = pd.read_csv(data_dir + "1a2k_k3_df.csv", engine= 'python')

df_x = (df[["cmi.m$value","cc.m$value","cp.m$value","cp1.m$value", "cp2.m$value", "hcm.m$value", "rsa.m$value","scm.m$value", "ssp.m$value","ecc.m$value", "ecmi.m$value", "ecp.m$value", "ecp1.m$value", "ecp2.m$value", "ehcm.m$value", "ersa.m$value", "escm.m$value", "essp.m$value"]])

df_y = (df[["inf.m$value"]])

df_x.shape, df_y[df_y["inf.m$value"] == 1].shape

((24304, 18), (117, 1))

In [4]:
file_list = glob.glob(data_dir + "*.csv")

for file_name in file_list:
    
        df_temp = pd.read_csv(file_name)
        df_xn = (df_temp[["cmi.m$value","cc.m$value","cp.m$value","cp1.m$value", "cp2.m$value", "hcm.m$value", "rsa.m$value","scm.m$value", "ssp.m$value","ecc.m$value", "ecmi.m$value", "ecp.m$value", "ecp1.m$value", "ecp2.m$value", "ehcm.m$value", "ersa.m$value", "escm.m$value", "essp.m$value"]])
        df_yn = (df_temp[["inf.m$value"]])
        df_x = df_x.append(df_xn, ignore_index=True)
        df_y = df_y.append(df_yn, ignore_index=True)
        print("'%-40s' file read with %i True" %(file_name, df_temp[df_temp["inf.m$value"] == 1].shape[0]))

'C:/Users/Acer/Documents/kriti/random_forests/training_k3\1a2k_k3_df.csv' file read with 117 True
'C:/Users/Acer/Documents/kriti/random_forests/training_k3\1b6c_k3_df.csv' file read with 60 True
'C:/Users/Acer/Documents/kriti/random_forests/training_k3\1buh_k3_df.csv' file read with 118 True
'C:/Users/Acer/Documents/kriti/random_forests/training_k3\1e6e_k3_df.csv' file read with 161 True
'C:/Users/Acer/Documents/kriti/random_forests/training_k3\1e96_k3_df.csv' file read with 100 True
'C:/Users/Acer/Documents/kriti/random_forests/training_k3\1grn_k3_df.csv' file read with 108 True
'C:/Users/Acer/Documents/kriti/random_forests/training_k3\1gxd_k3_df.csv' file read with 141 True
'C:/Users/Acer/Documents/kriti/random_forests/training_k3\1h1v_k3_df.csv' file read with 83 True
'C:/Users/Acer/Documents/kriti/random_forests/training_k3\1h9d_k3_df.csv' file read with 168 True
'C:/Users/Acer/Documents/kriti/random_forests/training_k3\1he8_k3_df.csv' file read with 87 True
'C:/Users/Acer/Document

In [5]:
df_x

,cmi.m$value,cc.m$value,cp.m$value,cp1.m$value,cp2.m$value,hcm.m$value,rsa.m$value,scm.m$value,ssp.m$value,ecc.m$value,ecmi.m$value,ecp.m$value,ecp1.m$value,ecp2.m$value,ehcm.m$value,ersa.m$value,escm.m$value,essp.m$value
0,0.747425,0.642105,0.380208,0.970297,0.649485,0.842105,0.875000,0.865854,0.6,0.128695,0.405053,0.004341,0.307943,0.373581,0.111101,0.320395,0.136048,0.266667
1,0.460760,0.615789,0.276042,0.712871,0.855670,0.347368,0.250000,0.932662,0.4,0.301955,0.564696,0.185416,0.525674,0.575125,0.331798,0.432414,0.424396,0.430769
2,0.546943,0.642105,0.380208,0.970297,0.649485,0.842105,0.500000,0.865854,0.4,0.303993,0.395378,0.157781,0.347092,0.506431,0.260565,0.211100,0.408378,0.343590
3,0.181230,0.652632,0.231771,0.118812,0.721649,0.331579,0.000000,0.723754,0.4,0.263967,0.257656,0.186501,0.345390,0.331820,0.328608,0.196118,0.428089,0.343590
4,0.036292,0.310526,0.437500,0.346535,0.329897,1.000000,0.250000,0.865854,0.4,0.255629,0.149792,0.147869,0.242411,0.323196,0.265527,0.072863,0.432262,0.343590
5,0.233629,0.615789,0.244792,0.801980,0.876289,0.357895,0.000000,0.977731,0.4,0.254887,0.158299,0.135426,0.315745,0.433348,0.262780,0.065373,0.429426,0.343590
6,0.189211,0.615789,0.276042,0.712871,0.855670,0.347368,0.000000,0.932662,0.4,0.295099,0.180387,0.103812,0.296170,0.574520,0.187915,0.000000,0.428701,0.343590
7,0.088440,0.615789,0.244792,0.801980,0.876289,0.357895,0.000000,0.977731,0.4,0.295099,0.111024,0.103161,0.294894,0.599637,0.185878,0.000000,0.407695,0.343590
8,0.017652,0.615789,0.276042,0.712871,0.855670,0.347368,0.000000,0.932662,0.4,0.295099,0.041334,0.141793,0.389220,0.520200,0.230088,0.000000,0.302307,0.430769
9,0.009648,0.615789,0.424479,0.960396,0.969072,0.505263,0.000000,0.619300,0.6,0.329288,0.011859,0.216378,0.308936,0.439855,0.306636,0.065373,0.289961,0.528205


In [6]:
df_x1 = df_x[df_y['inf.m$value']==0] # Dataframe x-axis with all zeros
df_y1 = df_y[df_y['inf.m$value']==0] # Dataframe y-axis with all zeros

df_x2 = df_x[df_y['inf.m$value']==1] # Dataframe x-axis with all ones
df_y2 = df_y[df_y['inf.m$value']==1] # Dataframe y-axis with all ones

In [7]:
df_x1.shape, df_y1.shape, df_x2.shape, df_y2.shape

((1890174, 18), (1890174, 1), (5063, 18), (5063, 1))

In [8]:
# Choose random indices from df_x1.index with length = 5*length of df_x2

rind = np.random.choice(np.array(df_x1.index), size=5*len(df_x2))

In [9]:
rind[1]

1488599

In [10]:
df_x1.loc[rind]

,cmi.m$value,cc.m$value,cp.m$value,cp1.m$value,cp2.m$value,hcm.m$value,rsa.m$value,scm.m$value,ssp.m$value,ecc.m$value,ecmi.m$value,ecp.m$value,ecp1.m$value,ecp2.m$value,ehcm.m$value,ersa.m$value,escm.m$value,essp.m$value
1302668,0.494022,0.500000,0.343750,0.544554,0.989691,0.836842,0.046875,0.522269,0.1,0.465106,0.443906,0.547805,0.599920,0.536221,0.764734,0.187551,0.674621,0.000000
1488599,0.354060,0.057895,0.335938,0.564356,0.092784,0.963158,0.562500,0.954931,0.3,0.238285,0.370950,0.506966,0.380891,0.649753,0.811043,0.391236,0.929853,0.400000
1459240,0.012102,0.510526,0.429688,0.207921,0.319588,0.736842,0.062500,0.954931,0.5,0.666886,0.325564,0.576819,0.436868,0.537681,0.553012,0.214337,0.888922,0.800000
1595412,0.252542,0.552632,0.393229,0.801980,0.319588,0.910526,0.375000,1.000000,0.5,0.541342,0.268018,0.530143,0.850720,0.793353,0.572642,0.405671,0.797701,0.800000
872988,0.037331,0.226316,0.341146,0.069307,0.659794,0.726316,0.468750,0.656946,0.5,0.527394,0.211229,0.522270,0.421924,0.494723,0.644392,0.422996,0.665046,0.864103
211033,0.002019,0.500000,0.439689,0.693069,0.432990,0.857895,0.000000,0.507423,0.1,0.633543,0.048411,0.516904,0.696485,0.591891,0.575732,0.000000,0.638217,0.000000
160839,0.016565,0.363158,0.356771,0.653465,0.835052,0.521053,0.093750,0.716331,0.6,0.526388,0.075645,0.477995,0.578167,0.798829,0.453437,0.325382,0.715181,1.000000
1439984,0.075030,0.631579,0.221354,0.881188,0.886598,0.463158,0.000000,0.843054,0.5,0.552167,0.154344,0.391491,0.810444,0.768564,0.488052,0.000000,0.819455,0.800000
1725817,0.437611,0.310526,0.437500,0.346535,0.329897,1.000000,0.428571,0.865854,0.5,0.650756,0.303770,0.726505,0.241281,0.365656,0.681744,0.431898,0.952752,0.800000
1079551,0.004779,0.473684,0.669261,0.940594,0.752577,0.952632,0.000000,0.432821,0.4,0.754848,0.165320,0.594357,0.505265,0.719775,0.590280,0.014276,0.587490,0.471795


In [11]:
df_x3 = df_x1.loc[rind]
df_y3 = df_y1.loc[rind]

In [12]:
df_xc = pd.concat([df_x3, df_x2], ignore_index=True)
df_yc = pd.concat([df_y3, df_y2], ignore_index=True)

In [13]:
df_xc.shape, df_yc.shape

((30378, 18), (30378, 1))

In [14]:
x = (df_xc.as_matrix())
y = (df_yc.as_matrix())

C:\Users\Acer\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\Acer\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [15]:
# for i in tqdm(range(len(x))):
#     x[i,0] = dict_var12[x[i,0]]
#     x[i,1] = dict_var12[x[i,1]]

In [16]:
scaler = StandardScaler().fit(x)
rescaledX = scaler.transform(x)

In [17]:
y = y.ravel()

In [18]:
data_dir = "C:/Users/Acer/Documents/kriti/random_forests/"

In [19]:
rf_random = joblib.load("RF_random_grid_model_with_env_model_kernel_3.sav")

In [20]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#### Testing on "data_frames_01102018/3i4r_final_df.csv"

In [21]:
data_dir = "C:/Users/Acer/Documents/kriti/random_forests/test_k3/"
df = pd.read_csv(data_dir + "test_df.csv")

df_x3i4r = (df[["cmi.m$value","cc.m$value","cp.m$value","cp1.m$value", "cp2.m$value","hcm.m$value","rsa.m$value",
            "scm.m$value","ssp.m$value", "ecc.m$value", "ecmi.m$value", "ecp.m$value", "ecp1.m$value", "ecp2.m$value", "ehcm.m$value", "ersa.m$value", "escm.m$value", "essp.m$value"]])



In [22]:
xtest_3i4r = np.array(df_x3i4r)


In [23]:
rsc_xtest_3i4r = scaler.transform(xtest_3i4r)

In [ ]:
ytest_3i4r = ytest_3i4r.ravel()

In [24]:
ypred_3i4r = rf_random.predict(rsc_xtest_3i4r)

In [ ]:
test_prob = rf_random.predict_proba(rsc_xtest_3i4r)

In [ ]:
sns.distplot(test_prob[(ytest_3i4r == 0) & (ypred_3i4r == 1)][:,1])
sns.distplot(test_prob[(ytest_3i4r == 1) & (ypred_3i4r == 1)][:,1])
plt.show()

In [25]:
threshold=0.5
predicted_proba = rf_random.predict_proba(rsc_xtest_3i4r)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

In [ ]:
predicted.shape, ytest_3i4r.shape

In [ ]:
cm = confusion_matrix(ytest_3i4r, predicted)#ypred_3i4r)

fig, ax = plt.subplots(figsize=(10, 8))
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
cmap = sns.cubehelix_palette(light=0.98, as_cmap=True)
unsuv_cm = pd.DataFrame(cm)
sns.heatmap(unsuv_cm, annot=True,annot_kws={"size": 16}, cmap=cmap, fmt='d')# font size
plt.xlabel("Predicted Class", fontsize=20, labelpad=20)
plt.ylabel("True Class", fontsize=20, labelpad=20)
plt.show()

In [ ]:
print(classification_report(ytest_3i4r, predicted))#ypred_3i4r))

In [26]:
predicted = pd.DataFrame(predicted, columns=['prediction']).to_csv('prediction_test.csv')